https://www.drugs.com/alpha/a.html

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

# params = {
#     'ie': 'UTF-8',
#     'wd': 'Python'
# }


In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://www.rxlist.com/supplements/alpha_a.htm', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，s
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

In [9]:
def articlehtml_download(headers_pc, randomproxy, cookie_dict, item, mislist):
    # pbar.set_description(item['term'])
    try:
        url = item['url']
        term = item['term'].strip().replace('/', ' ')
        if os.path.exists(f"D:/Intern_Project/9_drugs/article_htmls/{term}.html"):
#         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html"):
#             pbar.update(1)    
            return None 
#         time.sleep(random.randint(0, 5))
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
        
        with open(f"D:/Intern_Project/9_drugs/article_htmls/{term}.html", 'w+', encoding='utf-8') as f:
#         with open(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html", 'w+', encoding='utf-8') as f:
            f.write(html)
    
#         pbar.update(1)
    except Exception as e:
        print(e)
        headers_pc, randomproxy, cookie_dict = changeparams()
        mislist.append(item)
    return None

In [7]:
with open('D:/Intern_Project/9_drugs/术语链接列表.json', 'r', encoding='utf-8') as f:
    url_list = json.load(f)

print(len(url_list))


1020


In [10]:
mislist = []
    
count = 0
headers_pc, randomproxy, cookie_dict = changeparams()

for item in url_list:
    articlehtml_download(headers_pc, randomproxy, cookie_dict, item, mislist)
    count += 1
    if count % 100 == 0:
        # time.sleep(2)
        headers_pc, randomproxy, cookie_dict = changeparams()
        print(count)
        


100
200
300
400
500
600
700
800
900
1000


In [11]:
print(mislist)

[]
